# Welcome to my Tutorial for Artificial Neural Network
# Artificial Neural Network

# Getting more underestanding about our dataset:
* This is a dataset for a bank which collects information about the customers
* RowNumber: is an irrelevent feature that we are going to get rid of it in the future 
* Customerid: is the identification key of each customer
* Surname: is the surname of each customer
* CreditScore: is the credit score of each customer
* Geography: means the conutry which each customer lives in
* Gender: is the gender of each customer
* Age: is the age of each customer
* Tenure:means the number of the year(s) which customer in the bank
* Balance: means the amount of money that each customer has in their account
* NumOfProducts: the number of the product(s) which customer uses like Credict card, Master Card Etc,
* HasCrCard: means if the customer has the credit card, 1 means Yes and 0 means No
* isActiveMemebr: means if the customer is active or not which 1 means is a active customer
* EstimatedSalary: means the estimated salary of each customer
# Target
* Exited: means if the customer exists the bank or not, 1 means the customer left the bank 

# First, we are going to import the libraries


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
tf.__version__

Great! we have tensorflow 2 and this works much better than tensorflow 1 

# Let's import the dataset into the kernel

## Part 1 - Data Preprocessing

In [ ]:
dataset = pd.read_csv('../input/churn-predictions-personal/Churn_Predictions.csv')
X = dataset.iloc[:, 3:-1].values #we select all th features except RowNumber, Customerid, Surname, Exited
y = dataset.iloc[:, -1].values #we select the target feature Exited for the y

In [ ]:
dataset.head()

In [ ]:
print(X)

In [ ]:
print(y)

### Encoding categorical data

# Label Encoding the "Gender" column

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [ ]:
print(X)

Great, we encoded the Gender feature, we have 0 code for Female category

* One Hot Encoding the "Geography" column

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [ ]:
print(X)

Great, we encoded the Geography Column

### Splitting the dataset into the Training set and Test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Feature Scaling
* Notice: This is a very important and fundamental step for implementing the Deep Learning 

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [ ]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the second hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the output layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [ ]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

Use our ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer?

In [ ]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

### Predicting the Test set results

In [ ]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

### Making the Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)